In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import time
import threading
import requests 
from bs4 import BeautifulSoup
import json

#Data consent function
def consent():
    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    We wish to record your response data
    to an anonymised public data repository.
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.
    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("Please type yes to consent: ")

    if result.lower() == "yes":
        print("Thanks for your participation.")
        print("Please contact philip.lewis@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
    else:
        raise(Exception("User did not consent to continue test."))

#Username instructions
def username_instructions():
    id_instructions = """
    Enter your anonymised username
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifier would be CBTC
    """
    print(id_instructions)


game_rounds = []
total_rounds = 150
buttons_enabled = False
username = ""
enable_time = None  
current_delay = 0  

#Function to get username
def get_username():
    global username
    username = input("Enter your anonymous username: ")
    print("User entered id:", username)

#Function to generate dots within an oval
def generate_circles_in_oval(oval_coords, num_dots):
    dots = []
    x0, y0, x1, y1 = oval_coords
    width = x1 - x0
    height = y1 - y0
    margin = 0.1

    for _ in range(num_dots):
        angle = random.uniform(0, 2 * math.pi)
        radius_x = (width - margin) / 2 * random.uniform(0.6, 1)
        radius_y = (height - margin) / 2 * random.uniform(0.6, 1)
        x = x0 + width / 2 + radius_x * math.cos(angle)
        y = y0 + height / 2 + radius_y * math.sin(angle)
        dots.append((x, y))
    return dots

#Function to draw the game interface
def draw_game(left_dots, right_dots):
    fig, ax = plt.subplots()
    left_oval = Ellipse((0.3, 0.5), 0.4, 0.9, fill=False)
    right_oval = Ellipse((0.7, 0.5), 0.4, 0.9, fill=False)
    ax.add_patch(left_oval)
    ax.add_patch(right_oval)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    for x, y in left_dots:
        ax.plot(x, y, 'ro')
    for x, y in right_dots:
        ax.plot(x, y, 'bo')

    plt.close(fig)
    return fig

#Function called when a button is clicked
def on_button_click(button_label):
    global buttons_enabled
    global current_delay  
    if buttons_enabled:
        left_dots, right_dots = current_round_data
        correct = (button_label == "Left" and len(left_dots) > len(right_dots)) or \
                  (button_label == "Right" and len(right_dots) > len(left_dots))
        game_rounds.append({
            "Round": len(game_rounds) + 1,
            "Left Dots": len(left_dots),
            "Right Dots": len(right_dots),
            "Choice": button_label,
            "Correct": correct,
            "Username": username,
            "Delay": current_delay 
        })

        #Disable buttons immediately after a choice is made
        disable_buttons()
        next_round_button.disabled = False

#Function to enable buttons
def enable_buttons():
    global buttons_enabled
    global enable_time
    left_button.disabled = False
    right_button.disabled = False
    buttons_enabled = True
    enable_time = time.time()

#Function for actions after timer
def post_timer_actions():
    global enable_time
    current_time = time.time()
    if current_time - enable_time >= 3:
        disable_buttons()
        next_round_button.disabled = False

#Function to disable buttons
def disable_buttons():
    global buttons_enabled
    left_button.disabled = True
    right_button.disabled = True
    buttons_enabled = False

#Function to prepare the next round of the game
def prepare_next_round(_btn=None):
    global current_round_data
    global buttons_enabled
    global game_rounds
    global current_delay  
    buttons_enabled = False
    next_round_button.disabled = True

    if len(game_rounds) >= total_rounds:
        global game_data_df
        game_data_df = pd.DataFrame(game_rounds)
        send_data_to_google_form()

        with output:
            clear_output(wait=True)
            print("End of the Game! Data Sent Successfully")
            display(game_data_df)
        return

    with game_view:
        clear_output(wait=True)

    ratios = [(4, 3), (7, 6), (9, 8), (10, 9), (3, 4), (6, 7), (8, 9), (9, 10)]
    ratio = random.choice(ratios)
    base_number = random.randint(9, 21)
    left_num_dots = base_number
    right_num_dots = int(base_number * ratio[1] / ratio[0])

    left_dots = generate_circles_in_oval((0.1, 0.1, 0.5, 0.9), left_num_dots)
    right_dots = generate_circles_in_oval((0.5, 0.1, 0.9, 0.9), right_num_dots)
    current_round_data = (left_dots, right_dots)

    with game_view:
        display(draw_game(left_dots, right_dots))
    

    time.sleep(current_delay)  

    with game_view:
        clear_output(wait=False)
        enable_buttons()


#I could also put this inside the prepare_next_round function instead to give random value at each round
current_delay = random.choice([0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9, 1, 1.1, 1.2, 1.25, 1.3, 1.4, 1.5, 1.6, 1.7, 1.75, 1.8, 1.9, 2.0, 2.1, 2.25, 2.3, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4]) 


def send_data_to_google_form():
    game_data_csv = game_data_df.to_csv(index=False)
    data_dict = {'answers': game_data_csv}
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSewwYrqYPCLuTqPtiLkWJaj-tSc234CEn83zcB7Ee6G0MZ9Ag/viewform?usp=sf_link'
    
    def send_to_google_form(data_dict, form_url):
        ''' Helper function to upload information to a corresponding google form 
            You are not expected to follow the code within this function!
        '''
        form_id = form_url[34:90]
        view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
        post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

        page = requests.get(view_form_url)
        content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
        content = content.text[27:-1]
        result = json.loads(content)[1][1]
        form_dict = {}

        loaded_all = True
        for item in result:
            if item[1] not in data_dict:
                print(f"Form item {item[1]} not found. Data not uploaded.")
                loaded_all = False
                return False
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok

    send_to_google_form(data_dict, form_url)

#Function to periodically check if buttons should be disabled
def periodic_check():
    while True:
        if buttons_enabled:
            post_timer_actions()
        time.sleep(0.1)  # Check every 0.1 seconds

#Start the periodic check in a separate thread
check_thread = threading.Thread(target=periodic_check, daemon=True)
check_thread.start()

#Main execution
consent()
username_instructions()
get_username()

#Setup UI elements
left_button = widgets.Button(description="Left", disabled=True)
right_button = widgets.Button(description="Right", disabled=True)
next_round_button = widgets.Button(description="Next Round", disabled=False)
output = widgets.Output()
game_view = widgets.Output()

#Setup event handlers
left_button.on_click(lambda btn, _btn=left_button: on_button_click("Left"))
right_button.on_click(lambda btn, _btn=right_button: on_button_click("Right"))
next_round_button.on_click(prepare_next_round)

#Display UI elements
display(widgets.HBox([left_button, right_button, next_round_button]))
display(output)
display(game_view)

#Initialize and start the game
current_round_data = (None, None)
prepare_next_round(None)



DATA CONSENT INFORMATION:

    Please read:

    We wish to record your response data
    to an anonymised public data repository.
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.
    Please type yes in the box below if you consent to the upload.
Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.

    Enter your anonymised username
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifier would be CBTC
    
User entered id: ghgh


Output()

Output()